In [3]:
import numpy as np

# --- Day 8: Treetop Tree House ---

## Part 1

The expedition comes across a peculiar patch of tall trees all planted carefully in a grid. The Elves explain that a previous expedition planted these trees as a reforestation effort. Now, they're curious if this would be a good location for a tree house.

First, determine whether there is enough tree cover here to keep a tree house hidden. To do this, you need to count the number of trees that are visible from outside the grid when looking directly along a row or column.

The Elves have already launched a quadcopter to generate a map with the height of each tree (your puzzle input). For example:

30373
25512
65332
33549
35390

Each tree is represented as a single digit whose value is its height, where 0 is the shortest and 9 is the tallest.

A tree is visible if all of the other trees between it and an edge of the grid are shorter than it. Only consider trees in the same row or column; that is, only look up, down, left, or right from any given tree.

All of the trees around the edge of the grid are visible - since they are already on the edge, there are no trees to block the view. In this example, that only leaves the interior nine trees to consider:

    The top-left 5 is visible from the left and top. (It isn't visible from the right or bottom since other trees of height 5 are in the way.)
    The top-middle 5 is visible from the top and right.
    The top-right 1 is not visible from any direction; for it to be visible, there would need to only be trees of height 0 between it and an edge.
    The left-middle 5 is visible, but only from the right.
    The center 3 is not visible from any direction; for it to be visible, there would need to be only trees of at most height 2 between it and an edge.
    The right-middle 3 is visible from the right.
    In the bottom row, the middle 5 is visible, but the 3 and 4 are not.

With 16 trees visible on the edge and another 5 visible in the interior, a total of 21 trees are visible in this arrangement.

Consider your map; how many trees are visible from outside the grid?

In [32]:
forest_map=[]

with open('input.txt') as f:
    for line in f:
        if line.strip() != "":
            input = line.strip()
            forest_map.append(np.array([*input]).astype(int))

forest_map = np.array(forest_map)

visible = []

for i in range(forest_map.shape[0]):
    visible.append((i,0))
    col_index = 1
    last_height = forest_map[i,0]
    while col_index < forest_map.shape[1]:
        current_height = forest_map[i,col_index]
        if current_height> last_height:
            visible.append((i,col_index))
            last_height = current_height
        col_index += 1

    visible.append((i,forest_map.shape[1]-1))
    rev_col_index = forest_map.shape[1]-2
    last_height = forest_map[i,forest_map.shape[1]-1]
    while rev_col_index >= 0:
        current_height = forest_map[i,rev_col_index]
        if current_height> last_height:
            visible.append((i,rev_col_index))
            last_height = current_height
        rev_col_index -= 1
    
for j in range(forest_map.shape[1]):
    visible.append((0,j))
    row_index = 1
    last_height = forest_map[0,j]
    while row_index < forest_map.shape[0]:
        current_height = forest_map[row_index,j]
        if current_height> last_height:
            visible.append((row_index,j))
            last_height = current_height
        row_index += 1

    visible.append((forest_map.shape[0]-1,j))
    rev_row_index = forest_map.shape[0]-2
    last_height = forest_map[forest_map.shape[0]-1,j]
    while rev_row_index >= 0:
        current_height = forest_map[rev_row_index,j]
        if current_height> last_height:
            visible.append((rev_row_index,j))
            last_height = current_height
        rev_row_index -= 1
    
visible_location = list(set(visible))
result = np.ones_like(forest_map)
for loc_pair in visible_location:
    result[loc_pair[0],loc_pair[1]] = 0

with open('output.txt', 'w') as f:
    for r in result:
        f.write(" ".join(np.array(r).astype(str))+"\n")

print(len(visible_location))


1840


# Part 2

Content with the amount of tree cover available, the Elves just need to know the best spot to build their tree house: they would like to be able to see a lot of trees.

To measure the viewing distance from a given tree, look up, down, left, and right from that tree; stop if you reach an edge or at the first tree that is the same height or taller than the tree under consideration. (If a tree is right on the edge, at least one of its viewing distances will be zero.)

The Elves don't care about distant trees taller than those found by the rules above; the proposed tree house has large eaves to keep it dry, so they wouldn't be able to see higher than the tree house anyway.

In the example above, consider the middle 5 in the second row:

30373
25512
65332
33549
35390

    Looking up, its view is not blocked; it can see 1 tree (of height 3).
    Looking left, its view is blocked immediately; it can see only 1 tree (of height 5, right next to it).
    Looking right, its view is not blocked; it can see 2 trees.
    Looking down, its view is blocked eventually; it can see 2 trees (one of height 3, then the tree of height 5 that blocks its view).

A tree's scenic score is found by multiplying together its viewing distance in each of the four directions. For this tree, this is 4 (found by multiplying 1 * 1 * 2 * 2).

However, you can do even better: consider the tree of height 5 in the middle of the fourth row:

30373
25512
65332
33549
35390

    Looking up, its view is blocked at 2 trees (by another tree with a height of 5).
    Looking left, its view is not blocked; it can see 2 trees.
    Looking down, its view is also not blocked; it can see 1 tree.
    Looking right, its view is blocked at 2 trees (by a massive tree of height 9).

This tree's scenic score is 8 (2 * 2 * 1 * 2); this is the ideal spot for the tree house.

Consider each tree on your map. What is the highest scenic score possible for any tree?

In [41]:
tree_scenary_score = np.zeros_like(forest_map)

for i in range(forest_map.shape[0]):
    for j in range(forest_map.shape[1]):
        height = forest_map[i,j]
        score = {
            "up": 0,
            "down": 0,
            "left": 0,
            "right": 0
        }
        
        for up in range(i-1,-1,-1):
            if forest_map[up,j]>= height:
                score["up"] += 1
                break
            else:
                score["up"] += 1

        for down in range(i+1,forest_map.shape[1]):
            if forest_map[down,j]>= height:
                score["down"] += 1
                break
            else:
                score["down"] += 1
        
        for left in range(j-1,-1,-1):
            if forest_map[i,left]>= height:
                score["left"] += 1
                break
            else:
                score["left"] += 1

        for right in range(j+1,forest_map.shape[0]):
            if forest_map[i,right]>= height:
                score["right"] += 1
                break
            else:
                score["right"] += 1

        tree_scenary_score[i,j] = score["up"] * score["down"] * score["left"] * score["right"]

with open('output2.txt', 'w') as f:
    for r in tree_scenary_score:
        f.write("".join([f"{s:5}" for s in np.array(r).astype(str)])+"\n")

print(tree_scenary_score.max())

405769
